# **Базовый уровень: Генерация текста с использованием LSTM**
**Сбор данных:** используйте готовый или соберите свой корпус в формате plain text для генерации текстов

**Генерация текста на основе небольшого датасета**


*   Предварительный анализ: чистка текста
*   Обучение модели. Используйте образец из туториала по RNNи
*   Генерация текста. Используйте образец из туториала по RNN
*   Сгенерируйте несколько текстов с помощью созданной модели

**О данных**

Сообщения руководителя отдела СММ, которые сами по себе обладают уникальностей стилистикой

In [2]:
# Загрузка библиотек
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import json
import string
import random

## **Корпус**
Эта часть кода обрабатывает json-файл после экспорта чата. Остаются непосредственно только сообщения от нужного пользователя

In [20]:
def extract_filtered_text(
    input_json_path: str,
    output_txt_path: str,
    author_name: str = "руслан гуряшин"
):
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    with open(output_txt_path, 'w', encoding='utf-8') as out_f:
        for msg in data.get("messages", []):
            # Только сообщения от пользователей
            if msg.get("type") != "message":
                continue

            # Только от нужного пользователя
            if msg.get("from", "").lower() != author_name.lower():
                continue

            # Пропуск пересланных сообщений
            if "forwarded_from" in msg:
                continue

            # Извлечение текста
            raw_text = msg.get("text")
            if raw_text is None:
                continue

            if isinstance(raw_text, str):
                text = raw_text
            elif isinstance(raw_text, list):
                parts = []
                for part in raw_text:
                    if isinstance(part, dict) and "text" in part:
                        parts.append(part["text"])
                    elif isinstance(part, str):
                        parts.append(part)
                text = "".join(parts)
            else:
                continue

            text = text.replace("\n", " ").strip()
            if not text:
                continue
            if "https://" in text:
                continue
            out_f.write(text + "\n")


if __name__ == "__main__":
    extract_filtered_text(
        input_json_path="/content/result.json",
        output_txt_path="/content/ruslan.txt",
        author_name="руслан гуряшин"
    )

In [21]:
def filter_lines_by_word_count(input_txt_path: str, output_txt_path: str, min_words: int = 5):
    with open(input_txt_path, 'r', encoding='utf-8') as infile, \
         open(output_txt_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            words = line.strip().split()
            if len(words) >= min_words:
                outfile.write(" ".join(words) + "\n")


if __name__ == "__main__":
    filter_lines_by_word_count(
        input_txt_path="ruslan.txt",
        output_txt_path="ruslan5s.txt",
        min_words=5
    )

## **Работа с моделью**

In [3]:
# Загрузка данных
with open('/content/ruslan.txt', 'r', encoding='utf-8') as f:
    dataset = f.read().splitlines()

translator = str.maketrans('', '', string.punctuation)

def clean_text(s: str) -> str:
    s = s.lower()
    s = s.translate(translator)
    return ' '.join(s.split())

In [4]:
data = dataset[:8000]

In [5]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

X[:10], y[:10]

([[834],
  [834, 1458],
  [834, 1458, 103],
  [834, 1458, 103, 511],
  [834, 1458, 103, 511, 295],
  [834, 1458, 103, 511, 295, 14],
  [2416],
  [2416, 46],
  [1459],
  [14]],
 [1458, 103, 511, 295, 14, 835, 46, 2417, 2418, 41])

In [6]:
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [7]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Обучаем модель
history = model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 390s 2s/step - accuracy: 0.0180 - loss: 8.1727 - val_accuracy: 0.0300 - val_loss: 7.9863
Epoch 2/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - accuracy: 0.0248 - loss: 7.3700 - val_accuracy: 0.0284 - val_loss: 8.2203
Epoch 3/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 434s 2s/step - accuracy: 0.0265 - loss: 7.0833 - val_accuracy: 0.0297 - val_loss: 8.4246
Epoch 4/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.0316 - loss: 6.9203 - val_accuracy: 0.0305 - val_loss: 8.5694
Epoch 5/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.0299 - loss: 6.7481 - val_accuracy: 0.0297 - val_loss: 8.6932
Epoch 6/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 441s 2s/step - accuracy: 0.0354 - loss: 6.5751 - val_accuracy: 0.0331 - val_loss: 8.8019
Epoch 7/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - accuracy: 0.0385 - loss: 6.3985 - val_accuracy: 0.0331 - val_loss: 8.8904
Epoch 8/20
242/242 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 0.0397 - loss: 6.2353 - val_accu

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 593, 100)       │       687,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6870)           │     1,037,370 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,624,912 (21.46 MB)

 Trainable params: 1,874,970 (7.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,749,942 (14.30 MB)

In [12]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("хост", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
хост не понимаю что это ок уже с китайцами по местному
